<a href="https://colab.research.google.com/github/AdhirajChaddha/Kaggle-Titanic-Callenge/blob/master/Titanic_Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
testURL = 'https://raw.githubusercontent.com/AdhirajChaddha/Kaggle-Titanic-Callenge/master/test.csv'
trainURL = 'https://raw.githubusercontent.com/AdhirajChaddha/Kaggle-Titanic-Callenge/master/train.csv'

trainAndTest = pd.read_csv(trainURL)
predict = pd.read_csv(testURL)

train_test_split = 0.2
length = len(trainAndTest)
split = int(train_test_split*length)

test, train =  trainAndTest.iloc[:split, :], trainAndTest.iloc[split:, :]

In [89]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
178,179,0,2,"Hale, Mr. Reginald",male,30.0,0,0,250653,13.0000,NaN,S
179,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0000,NaN,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.5500,NaN,S
181,182,0,2,"Pernot, Mr. Rene",male,NaN,0,0,SC/PARIS 2131,15.0500,NaN,C
182,183,0,3,"Asplund, Master. Clarence Gustaf Hugo",male,9.0,4,2,347077,31.3875,NaN,S


In [0]:
# This function removes unnecessary coulmns and returns the cleaned up dataframe
def clean(data):
  # We dont need the coulmn for the name as it does not have any relation
  data = data.drop(columns=['Name','Cabin', 'Ticket'])
  # Male = 1, Female = 0
  data.Sex.replace(['male', 'female'], [1, 0], inplace=True)
  # for Embarked, S = 0, C = 1, Q = 2
  data.Embarked.replace(['S', 'C', 'Q'], [0, 1, 2], inplace=True)
  return data

In [91]:
# Get the number of empty values in the data
def printOutNullItems(trainData, tag=""):
  print(tag)
  for (columnName, columnData) in trainData.iteritems():
    print(columnName , ":", trainData[columnName].isna().sum())

printOutNullItems(train)


PassengerId : 0
Survived : 0
Pclass : 0
Name : 0
Sex : 0
Age : 142
SibSp : 0
Parch : 0
Ticket : 0
Fare : 0
Cabin : 544
Embarked : 1


In [0]:
# Lets now make 2 copies of the data one where the null age gaps and removed 
# from the dataset, and one where it if filled with data using a normal 
# distrubution where with mean and std dev of all people on board. 
from scipy import stats
import numpy as np

# This funtion takes in a dataframe and reomoves NaN data rows
def removeNaN(data):
  train_remove = data.dropna(subset=["Age", "Embarked"])
  return train_remove

# This funtion takes in a dataframe and fills NaN data rows with normal distribution
def normalizeNaN(data):
  data_copy = data.copy()
  # Remove all rows with null items
  train_remove = data.dropna(subset=["Age", "Embarked"])

  # Get the STd Dev and mean for normal distribution of age
  ageSTDEV = train_remove.loc[:,"Age"].std()
  ageMean = scipy.stats.gmean(train_remove.loc[:,"Age"])

  # Drop all the rows in normal where Embarked is NaN
  train_normal = data_copy.dropna(subset=["Embarked"])
  train_normal["Age"].fillna(np.random.normal(ageMean, ageSTDEV), inplace = True) 

  return train_normal

In [0]:
# This function devides the dataframe into labels and features and returns both
def getDataAndLabels(data):
  labels = data[['Survived']]
  features = data.drop(columns=['Survived', "PassengerId"])

  return features, labels

In [0]:
# Function that takes a dataframe and normalize bool as args and then returns the
# cleaned up features and labels for the data
def getData(df, normalize):
  df = clean(df)

  if (normalize):
    df = normalizeNaN(df)
  else:
    df = removeNaN(df)
  
  features, labels = getDataAndLabels(df)
  return features, labels

In [95]:
features_normal, labels_normal = getData(train, True)
features_removed, labels_removed = getData(train, False)

features_test, labels_test = getData(test, False)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [97]:
print("features_normal: ", features_normal.shape)
print("labels_normal: ", labels_normal.shape)

print("features_removed: ", features_removed.shape)
print("labels_removed: ", labels_removed.shape)

print("features_test: ", features_test.shape)
print("labels_test: ", labels_test.shape)

features_normal.head()

features_normal:  (712, 7)
labels_normal:  (712, 1)
features_removed:  (570, 7)
labels_removed:  (570, 1)
features_test:  (142, 7)
labels_test:  (142, 1)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
178,2,1,30.000000,0,0,13.0000,0.0
179,3,1,36.000000,0,0,0.0000,0.0
180,3,0,44.853356,8,2,69.5500,0.0
181,2,1,44.853356,0,0,15.0500,1.0
182,3,1,9.000000,4,2,31.3875,0.0


In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(features_normal, labels_normal)

prediction = random_forest.predict(features_test)

accuracy = accuracy_score(prediction, labels_test)

print("Accuracy is: ", accuracy)

Accuracy is:  0.7605633802816901


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [101]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(features_removed, labels_removed)

prediction = random_forest.predict(features_test)

accuracy = accuracy_score(prediction, labels_test)

print("Accuracy is: ", accuracy)

Accuracy is:  0.7394366197183099


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
